### Testing specific issues with the CV's
Some testing... Sara's failed due to the "ć"

In [201]:
import pdfplumber
with pdfplumber.open("C:\Projects\Staffing\data\CV's\Sara_Vera_Marjanovi_.pdf") as pdf:
    print(pdf.pages[0].extract_text()[:-60])
    #for j, page in enumerate(pdf.pages):
    #    print(page.extract_text())

Sara Vera Marjanović
Data Scientist
Data scientist
Sara is a driven data scientist with a particular specialty in Machine Learning, Deep learning, and Natural Language 
Processing. She has end-to-end experience with wrangling a wide variety of different data sources (from free text to 
biometric data) to create either simple (e.g. clustering, regression) and complex models (e.g. Generative Adversarial 
Models) in the production pipeline. Therefore, she is well-versed in ideation, data collection, data wrangling and 
exploration, as well as model development and trouble-shooting and is comfortable with Python (especially the libraries 
Numpy, Pandas, Tensorflow, Pytorch) as well as SQL, R, and Azure DevOps.
Within her model creation and data exploration, Sara takes special care to limit bias within the dataset and model output, 
and to ensure interpretability of the results. She is diligent and conscientious with her data storytelling and data 
visualization, making her a skilled commun

### Extracting text as string from pdf', save to .csv
Here we read the text from all CV's and save the texts in a .csv file. Each line in the .csv contains name of the colleague, the page number of the CV and the text on that specific page. 

In [202]:
import csv
import glob
import regex as re
import pdfplumber
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Get list of CV paths
paths = glob.glob("C:\Projects\Staffing\data\CV's\*.pdf")
#print(paths)

stop_words = set(stopwords.words('english'))

with open("cv_test.csv", "w", newline='') as f:
    writer = csv.writer(f)
    csv_input = ["colleague", "total_text", "word_tokens"]
    writer.writerows([csv_input])
    for i, path in enumerate(paths):
        print(f"Currently at path: {path[31:-4]}")
        with pdfplumber.open(path) as pdf:
            total_text = ''
            for j, page in enumerate(pdf.pages):

                text = page.extract_text()
                text = re.sub('[^A-Za-z0-9 .,]+', '', text)
                total_text += text
                #word_tokens = word_tokenize(text)
                #filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
                # Only special characters are removed.
                #csv_input = [path[31:-4],j+1, text, filtered_sentence]
                #Write page text to pdf
                #writer.writerows([csv_input])

            word_tokens = word_tokenize(total_text)
            filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
            csv_input = [path[31:-4], total_text, filtered_sentence]
            writer.writerows([csv_input])

Currently at path: Antonio_Arfe
Currently at path: Ashish_Meshram
Currently at path: Ayman_Eleya_Zaki
Currently at path: Aziz_Ari
Currently at path: Branislav_Machava
Currently at path: Daniel_Masson
Currently at path: Daria_Korzel
Currently at path: Devichand_Das
Currently at path: Dinesh_Kumar_Padala
Currently at path: Ditlev_J_rgensen
Currently at path: Drisya_Thumba
Currently at path: Emil_Djursner_Rasmussen
Currently at path: Eugenia_Borgia
Currently at path: Hamza_Minhas
Currently at path: Hatef_Abdollahi
Currently at path: Jacob_Karlstr_m
Currently at path: Jan_Neldeberg
Currently at path: Jayant_Pant
Currently at path: Jes_Melbye_Chergui
Currently at path: Julian_Eikhaug
Currently at path: Kristina_Kj_r_Hedegaard
Currently at path: Ksheeraja_Vinjanampati_
Currently at path: Lee_Ann_Zambrano_Larsen
Currently at path: Line_Bruun
Currently at path: Mohd_Arshad
Currently at path: Mohit_Kumar_Bhati
Currently at path: Mostafa_Ellabaan
Currently at path: Olivia_Essen
Currently at path

### Create TaggedDocument, train Doc2Vec model on data
First we created a TaggedDocument object where we tag each document with the respective name. Each document is a page on the CV.

In [203]:
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
from nltk.tokenize import word_tokenize
import pandas as pd

df = pd.read_csv("cv_test.csv")

tagged_data = []
tags = df["colleague"].unique().tolist()
#print(tags)
#cnt = 0
"""
if used on text
for tags in tags:
    docs = df[df["colleague"] == tags]["page_text"].tolist()
    for doc in docs:
        tagged_data.append(TaggedDocument(words=word_tokenize(doc), tags=[str(tags)]))
    
    #cnt += 1
"""

# if used on word tokens that are filtered
for tag in tags:
    #print(tag)
    text = df[df["colleague"] == tag]["total_text"]
    
    word_tokens = df[df["colleague"] == tag]["word_tokens"]
    #print(text)
    #print(word_tokens)
    #1/0
    word_tokens = word_tokenize(str(total_text))
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    tagged_data.append(TaggedDocument(words=filtered_sentence, tags=[str(tag)]))


#print(tagged_data[1])
#1/0
#tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(name)]) for name, _d in (df["colleague"], df["page_text"])]
max_epochs = 100
vec_size = 300
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=5,
                dm =1)

model.build_vocab(tagged_data) # tagged_data

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged_data
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v_cvs.model")
print("Model Saved")

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### Use trained model to find best match between a subject and a colleague.
Here we use the trained model to match which CV best fits the given text (subject tested on is Data Migration).

In [207]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy

abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
file_name = "data_migration.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\data_migration.txt
Match Rank: 0, colleague: Daniel_Masson, distance: 0.6444
Match Rank: 1, colleague: Rajitha_Dodda, distance: 0.6589
Match Rank: 2, colleague: Rasmus_Rosenmeyer_Poulsen, distance: 0.6619
Match Rank: 3, colleague: Antonio_Arfe, distance: 0.6633
Match Rank: 4, colleague: Viktor_Ekstr_m, distance: 0.665
Match Rank: 5, colleague: Olivia_Essen, distance: 0.6762


# subject testet on data science

In [205]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy

abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
file_name = "data_science.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\data_science.txt
Match Rank: 0, colleague: Ditlev_J_rgensen, distance: 0.4978
Match Rank: 1, colleague: Peter_Damm, distance: 0.499
Match Rank: 2, colleague: Antonio_Arfe, distance: 0.504
Match Rank: 3, colleague: Mohit_Kumar_Bhati, distance: 0.509
Match Rank: 4, colleague: Olivia_Essen, distance: 0.5184
Match Rank: 5, colleague: Aziz_Ari, distance: 0.5248


In [206]:
import regex as re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import sys
import os
from scipy import spatial
import numpy
model= Doc2Vec.load("d2v_cvs.model")
print("Model loaded!")

cvs = pd.read_csv("cv_test.csv")
print("CVs loaded!")
abspath = r"C:\Projects\Staffing\sample_texts"
stop_words = set(stopwords.words('english'))

file_name = "software_development_python.txt"
print(os.path.join(abspath,file_name))
with open(os.path.join(abspath,file_name)) as f:
    text = f.read().replace('\n', '')
    f.close()

text = re.sub('[^A-Za-z0-9 .,]+', '', text)
text = word_tokenize(text)
filtered_text_object = [w for w in text if not w.lower() in stop_words]
# calculate distances between all cvs and target text
distances = []
for cv in cvs["total_text"]:
    word_tokens = word_tokenize(cv)
    filtered_cv = [w for w in word_tokens if not w.lower() in stop_words]
    distances.append(model.wv.wmdistance(filtered_text_object, filtered_cv))


# print top 5 matches
s = numpy.array(distances)
sort_index = numpy.argsort(s)
cnt = 0
for ind in sort_index:
    if cnt <= 5:
        #print(cvs.iloc[ind]["colleague"])
        print(f"Match Rank: {cnt}, colleague: {cvs.iloc[ind]['colleague']}, distance: {numpy.round(distances[ind],4)}")
        cnt += 1

Model loaded!
CVs loaded!
C:\Projects\Staffing\sample_texts\software_development_python.txt
Match Rank: 0, colleague: Eugenia_Borgia, distance: 0.5227
Match Rank: 1, colleague: Jacob_Karlstr_m, distance: 0.5863
Match Rank: 2, colleague: Olivia_Essen, distance: 0.6213
Match Rank: 3, colleague: Daniel_Masson, distance: 0.6295
Match Rank: 4, colleague: Ayman_Eleya_Zaki, distance: 0.6305
Match Rank: 5, colleague: Sara_Vera_Marjanovi_, distance: 0.6413
